In [1]:
%%capture
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
import numpy as np
import scipy as sp

In [2]:
FULL_SET_PATH = './data/aan_full.json'
TEST_SET_PATH = './data/aan_test.json'
CONCAT_OUTPUT_PATH = './results/base_tfidfcosine_concat_aan.json'
SUM_OUTPUT_PATH = './results/base_tfidfcosine_sum_aan.json'

In [15]:
FULL_SET_PATH = './data/dblp_full.json'
TEST_SET_PATH = './data/dblp_test.json'
CONCAT_OUTPUT_PATH = './results/base_tfidfcosine_concat_dblp.json'
SUM_OUTPUT_PATH = './results/base_tfidfcosine_sum_dblp.json'

In [16]:
with open(FULL_SET_PATH) as f:
    full_set = json.load(f)

In [17]:
corpus = [ref['title'] + ' ' + ref['abstract'] for ref in full_set]
vectorizer = TfidfVectorizer(stop_words='english').fit(corpus)
print(len(vectorizer.get_feature_names()))

43194


In [18]:
for ref in tqdm(full_set):
    document = [ref['title'] + ' ' + ref['abstract']]
    ref['tfidf_vector'] = vectorizer.transform(document)

In [19]:
full_set_dict = dict([(paper['id'], paper) for paper in full_set])

In [20]:
paper_tfidf_vectors = [paper['tfidf_vector'] for paper in full_set]
paper_ids = [paper['id'] for paper in full_set]

paper_tfidf_array = sp.sparse.vstack(paper_tfidf_vectors)

In [21]:
with open(TEST_SET_PATH) as f:
    test_set = json.load(f)

# concat variant

In [22]:
input_documents = []
for input_papers in test_set:
    input_document = ''
    
    for input_paper in input_papers:
        input_document += input_paper['title'] + ' ' + input_paper['abstract']
    
    input_documents.append(input_document)

input_tfidf_array = vectorizer.transform(input_documents)

score_array = cosine_similarity(input_tfidf_array, paper_tfidf_array)

In [23]:
results = []

for input_papers, scores  in tqdm(zip(test_set, score_array.tolist())):
    result = {}
    result['input'] = [input_paper['id'] for input_paper in input_papers]
    input_paper_ids_set = set(result['input'])
    
    candidate_scores = list(zip(paper_ids, scores))     
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    filtered_candidate_scores = [cs for cs in candidate_scores if cs[0] not in input_paper_ids_set]
    
    result['output'] = [cs[0] for cs in filtered_candidate_scores[:100]]
    results.append(result)

In [24]:
with open(CONCAT_OUTPUT_PATH, 'w') as f:
    json.dump(results, f)

# sum variant

In [25]:
partial_score_arrays = []
for i in tqdm(range(3)):
    partial_input_ids = [input_papers[i]['id'] for input_papers in test_set]
    input_tfidf_vectors = [full_set_dict[paper_id]['tfidf_vector'] for paper_id in partial_input_ids]
    
    input_tfidf_array = sp.sparse.vstack(input_tfidf_vectors)

    partial_score_arrays.append(cosine_similarity(input_tfidf_array, paper_tfidf_array))

score_array = sum(partial_score_arrays)

In [26]:
results = []

for input_papers, scores  in tqdm(zip(test_set, score_array.tolist())):
    result = {}
    result['input'] = [input_paper['id'] for input_paper in input_papers]
    input_paper_ids_set = set(result['input'])
    
    candidate_scores = list(zip(paper_ids, scores))     
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    filtered_candidate_scores = [cs for cs in candidate_scores if cs[0] not in input_paper_ids_set]
    
    result['output'] = [cs[0] for cs in filtered_candidate_scores[:100]]
    results.append(result)

In [27]:
with open(SUM_OUTPUT_PATH, 'w') as f:
    json.dump(results, f)